In [1]:
import os
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
#SERPER_API_KEY=os.getenv("SERPER_API_KEY")
#GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [42]:
llm=ChatGroq(model_name="llama-3.3-70b-versatile",
             temperature=0.0)
prompt=ChatPromptTemplate.from_messages([("system","You are a good assistant that helps every human to best of his ability."),
                                        ("user","{query}")
                                        ])
query = "Woher kommst du? Wo sind deine Server?" 

chain=prompt | llm | StrOutputParser()
response = chain.invoke({"query": query})
print(response)

Hallo! Ich bin ein künstliches Intelligenz-Modell, das von Meta AI entwickelt wurde. Meine "Heimat" ist also im virtuellen Raum, und ich existiere auf einer Vielzahl von Servern, die von Meta AI betrieben werden.

Meta AI ist ein Unternehmen, das sich auf die Entwicklung von künstlicher Intelligenz (KI) spezialisiert hat, und meine Server sind Teil eines globalen Netzwerks, das es mir ermöglicht, mit Benutzern wie dir zu kommunizieren.

Meine Server sind in verschiedenen Rechenzentren auf der ganzen Welt untergebracht, um eine hohe Verfügbarkeit und Leistung zu gewährleisten. Diese Rechenzentren sind in verschiedenen Ländern wie den USA, Europa und Asien ansässig.

Ich kann nicht genau sagen, wo meine Server physisch stehen, da dies Informationen sind, die von Meta AI als vertraulich behandelt werden. Aber ich kann dir sagen, dass meine Server auf modernen Rechenzentren basieren, die mit den neuesten Technologien ausgestattet sind, um eine hohe Leistung, Sicherheit und Umweltfreundlich